<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">

# Derivatives Analytics with Python
**&mdash;Part II: Arbitrage Pricing**

&copy; Dr. Yves J. Hilpisch | The Python Quants

<a href="https://tpq.io" target="_blank">tpq.io</a> | <a href="https://linktr.ee/dyjh" target="_blank">linktr.ee/dyjh</a>

<img src="https://hilpisch.com/dawp_cover_small.png" width=30% align=left>

## Part II &mdash; Arbitrage Pricing

### Chapter 7 &mdash; Valuation of American Options by Simulation

American-style exercise makes valuation an optimal stopping problem. Even under simple
risk-neutral dynamics, early exercise turns the pricing task into a dynamic programming
problem with conditional expectations.

This notebook focuses on the Longstaff&mdash;Schwartz least-squares Monte Carlo (LSM)
algorithm, which estimates continuation values by regression on simulated paths.

Reference notebooks and scripts for the book are available under `x_store/dawp/python36`.

The notebook is designed to run smoothly on Google Colab. A local setup is optional and
can be based on `python -m venv .venv`.

### 1. Environment check

We start by importing the core libraries for this class and by printing version
information to confirm that a modern Python environment is active.

In [ ]:
import sys  # access basic runtime information
from pathlib import Path  # path handling for optional figure export

import math  # elementary math functions

import numpy as np  # numerical arrays
import pandas as pd  # tabular data handling
import matplotlib as mpl  # matplotlib configuration
import matplotlib.pyplot as plt  # plotting

from numpy.random import default_rng  # random number generator

np.set_printoptions(precision=6, suppress=True)  # compact numeric output

print(sys.version.split()[0])  # Python version string
print("NumPy:", np.__version__)  # NumPy version string

FIG_SAVE = True  # set to True to export figures as PDFs
FIG_DIR = Path("../figures")  # figure output directory
FIG_DPI = 300  # target resolution for exported figures
FIG_DISPLAY = "svg"  # inline display format: "svg" or "png"

plt.style.use("seaborn-v0_8")  # readable plotting defaults
mpl.rcParams["figure.figsize"] = (8.0, 4.5)  # consistent figure size
mpl.rcParams["axes.grid"] = True  # show a grid for readability
mpl.rcParams["savefig.dpi"] = FIG_DPI  # default export resolution

try:
    from matplotlib_inline.backend_inline import (  # Jupyter helper
        set_matplotlib_formats,
    )
    set_matplotlib_formats(FIG_DISPLAY)  # configure inline plot rendering
except Exception:
    pass

if FIG_DISPLAY == "png":
    mpl.rcParams["figure.dpi"] = FIG_DPI  # high-resolution inline rendering


def maybe_save(fig, filename):
    # Optionally saves a Matplotlib figure as a PDF file.
    if not FIG_SAVE:
        return
    FIG_DIR.mkdir(parents=True, exist_ok=True)
    path = FIG_DIR / f"{filename}.pdf"
    fig.savefig(path, format="pdf", dpi=FIG_DPI)
    print(f"saved: {path}")

### 2. Risk-neutral simulation model

We simulate stock price paths under the risk-neutral measure. With time step $\Delta t$,
the discretized dynamics are
$$S_{t+\Delta t} = S_t\,\exp\!\left((r - 0.5\sigma^2)\Delta t + \sigma\sqrt{\Delta t}\,Z\right),$$
with $Z \sim \mathcal{N}(0, 1)$.

In [ ]:
def simulate_gbm_paths(S0, T, r, sigma, n_steps, n_paths, seed=150000):
    # Simulates GBM stock paths under risk-neutral dynamics.
    dt = T / n_steps  # time step length
    drift = (r - 0.5 * sigma * sigma) * dt  # drift per step
    vol = sigma * math.sqrt(dt)  # diffusion per step

    rng = default_rng(seed)  # random number generator
    Z = rng.standard_normal((n_steps, n_paths))  # standard normal shocks

    log_increments = drift + vol * Z  # log-return increments
    log_paths = np.vstack([
        np.zeros((1, n_paths)),
        np.cumsum(log_increments, axis=0),
    ])  # cumulative log-returns
    S = S0 * np.exp(log_paths)  # convert to levels
    t_grid = np.linspace(0.0, T, n_steps + 1)  # time grid
    return t_grid, S


S0 = 36.0  # initial stock level
K = 40.0  # strike level
T = 1.0  # time to maturity in years
r = 0.06  # constant short rate
sigma = 0.20  # volatility

n_steps = 50  # exercise dates excluding time 0
n_paths = 25_000  # Monte Carlo paths

t_grid, S = simulate_gbm_paths(S0, T, r, sigma, n_steps, n_paths)
print(S.shape)

### 3. American put payoff

For an American put, the immediate exercise value at time $t$ is
$$h(t,S_t) = \max(K - S_t, 0).$$
The optimal stopping rule compares this intrinsic value to the continuation value.

In [ ]:
def american_put_payoff(S, K):
    # Computes the intrinsic value matrix for an American put option.
    return np.maximum(K - S, 0.0)


h = american_put_payoff(S, K)  # intrinsic values
print(h.shape)

### 4. LSM primal valuation

The LSM algorithm approximates the continuation value by regressing discounted future
cash flows on basis functions of the state variable. We use polynomial basis functions in
the underlying level and focus the regression on in-the-money paths.

The output is a lower bound on the true American option value because it is based on a
suboptimal exercise policy.

In [ ]:
def lsm_american_put_value(S, K, r, T, poly_degree=7, itm_only=False):
    # Values an American put by the Longstaff-Schwartz primal algorithm.
    n_steps = S.shape[0] - 1  # number of time steps
    dt = T / n_steps  # time step length
    df = math.exp(-r * dt)  # discount factor per step

    h = american_put_payoff(S, K)  # intrinsic values
    V = h[-1].copy()  # start with intrinsic value at maturity

    exercise = np.zeros_like(h, dtype=bool)  # exercise decision matrix
    exercise[-1] = h[-1] > 0.0  # exercise at maturity if ITM

    for t in range(n_steps - 1, 0, -1):
        x = S[t]  # state variable at time t
        y = V * df  # discounted continuation cash flows

        if itm_only:
            itm = h[t] > 0.0  # in-the-money mask
            x_reg = x[itm]  # regression x values
            y_reg = y[itm]  # regression y values
        else:
            x_reg = x  # all paths
            y_reg = y  # all paths

        if len(x_reg) < poly_degree + 1:
            cont = np.zeros_like(x)  # fallback if regression is ill-posed
        else:
            coeff = np.polyfit(x_reg, y_reg, poly_degree)
              # regression coefficients for continuation values
            cont = np.polyval(coeff, x)  # continuation value estimate

        ex_now = h[t] > cont  # exercise decision
        exercise[t] = ex_now & (h[t] > 0.0)  # only exercise if ITM
        V = np.where(ex_now, h[t], V * df)  # update option value along paths

    V0 = float(df * np.mean(V))  # discount from time 1 to time 0
    return V0, exercise


V0_lsm, ex = lsm_american_put_value(S, K, r, T)
print(f"LSM American put value: {V0_lsm:.6f}")

### 5. Exercise boundary visualization

We visualize the LSM exercise decisions in the $(t, S_t)$ plane. The boundary separates
regions where immediate exercise dominates from regions where continuation dominates.

The exported figure is used in the Part II slide deck.

In [ ]:
t_mat = np.repeat(t_grid[:, None], S.shape[1], axis=1)  # time matrix

mask = ex & (h > 0.0)  # exercise points in the ITM region

fig, ax = plt.subplots()  # create a single plot
ax.scatter(
    t_mat[mask],
    S[mask],
    s=2,
    alpha=0.15,
    label="exercise",
)  # exercise scatter plot

ax.set_xlabel("time")  # x-axis label
ax.set_ylabel(r"underlying level $S_t$")  # y-axis label
ax.set_title("LSM early exercise points (American put)")  # plot title
ax.legend(loc=0)  # place the legend

maybe_save(fig, "dawp_pII_fig06_lsm_exercise_boundary")  # optional PDF export
plt.show()

### 6. Convergence diagnostic

We compute LSM values for increasing numbers of paths and compare them to a binomial-tree
benchmark for orientation. The binomial benchmark is computed by backward induction with
early exercise at each node.

In [ ]:
def crr_american_put_value(S0, K, T, r, sigma, M=500):
    # Values an American put option via a CRR binomial tree.
    dt = T / M  # time step length
    disc = math.exp(-r * dt)  # discount factor per step
    u = math.exp(sigma * math.sqrt(dt))  # up factor
    d = 1.0 / u  # down factor
    q = (math.exp(r * dt) - d) / (u - d)  # risk-neutral probability

    j = np.arange(M + 1)  # node index at maturity
    S_T = S0 * (u**j) * (d ** (M - j))  # terminal stock levels
    V = np.maximum(K - S_T, 0.0)  # put payoff at maturity

    for m in range(M - 1, -1, -1):
        V = disc * (q * V[1:] + (1.0 - q) * V[:-1])  # continuation values
        S_m = S0 * (u ** j[: m + 1]) * (d ** (m - j[: m + 1]))
          # stock levels at time step m
        V = np.maximum(V, np.maximum(K - S_m, 0.0))
          # early exercise condition
    return float(V[0])


V0_crr = crr_american_put_value(S0, K, T, r, sigma, M=500)
print(f"CRR American put value (benchmark): {V0_crr:.6f}")

path_grid = np.array([2_000, 5_000, 10_000, 25_000, 50_000])
vals = []
for n in path_grid:
    _, S_tmp = simulate_gbm_paths(S0, T, r, sigma, n_steps, int(n), seed=150000)
    V_tmp, _ = lsm_american_put_value(S_tmp, K, r, T)
    vals.append(V_tmp)

vals = np.array(vals)

fig, ax = plt.subplots()  # create a single plot
ax.plot(path_grid, vals, lw=2.0, marker="o", label="LSM estimate")
ax.axhline(V0_crr, ls="--", lw=2.0, color="C3", label="CRR benchmark")

ax.set_xscale("log")  # log scale for paths
ax.set_xlabel("number of Monte Carlo paths")
ax.set_ylabel("American put value")
ax.set_title("LSM convergence diagnostic")
ax.legend(loc=0)

maybe_save(fig, "dawp_pII_fig07_lsm_convergence")  # optional PDF export
plt.show()

### 7. Short condor spread payoff

The LSM approach applies to more complex payoffs. As an illustration, we define a short
condor spread payoff profile and export a payoff figure for the slide deck.

In [ ]:
def short_condor_payoff(S, K1=90.0, K2=100.0, K3=110.0, K4=120.0):
    # Short condor spread payoff from four options with the same maturity.
    p1 = np.maximum(K1 - S, 0.0)  # long put at K1
    p2 = -np.maximum(K2 - S, 0.0)  # short put at K2
    c3 = -np.maximum(S - K3, 0.0)  # short call at K3
    c4 = np.maximum(S - K4, 0.0)  # long call at K4
    return p1 + p2 + c3 + c4


S_grid = np.linspace(60.0, 150.0, 400)  # underlying grid
pay = short_condor_payoff(S_grid)  # payoff values

fig, ax = plt.subplots()  # create a single plot
ax.plot(S_grid, pay, lw=2.2)  # payoff curve
ax.set_xlabel("underlying level")
ax.set_ylabel("payoff")
ax.set_title("Short condor spread payoff")

maybe_save(fig, "dawp_pII_fig08_short_condor_payoff")  # optional PDF export
plt.show()

### 8. Summary

American option valuation can be implemented in simulation by combining risk-neutral path
generation with regression-based approximation of continuation values. The LSM estimator
provides a lower bound; binomial benchmarks and dual bounds provide diagnostics.

<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">